In [1]:
import numpy as np
import itertools
from timeit import default_timer as timer

In [2]:
def get_input(name):
    with open(f'{name}.txt') as f:
        return f.read()

# Part 1

In [3]:
class Rock:
    rock_shapes = [[12, 13, 14, 15], # I, lying down
           [5, 8, 9, 10, 13], # plus
           [6, 10, 14, 13, 12], # backwards L
           [0, 4, 8, 12], # I, standing up
           [8, 9, 12, 13], # box
          ]
    rock_heights = [1, 3, 3, 4, 2]
    
    def __init__(self, y, rock_shape):
        self.x = 2
        self.y = y
        self.shape = rock_shape
    
    def max_y(self):
        return self.y + HITBOX_SIZE - self.rock_heights[self.shape]
         
    def image(self):
        return self.rock_shapes[self.shape]

In [99]:
class Cave:
    n_rocks = 0
    state = 'active'
    height_outside_of_view = 0
    
    
    def __init__(self, height, width, rock_limit, jet_streams):
        self.height = height
        self.current_height = height
        self.width = width
        self.field = np.zeros([height, width])
        self.rock_limit = rock_limit
        self.rock_cycle = itertools.cycle([0, 1, 2, 3, 4])
        self.jet_cycle = itertools.cycle(jet_streams)
        
        
    def intersects(self):
        intersection = False
        for i in range(HITBOX_SIZE):
            for j in range(HITBOX_SIZE):
                if i * HITBOX_SIZE + j in self.rock.image():
                    if i + self.rock.y > self.height - 1 or \
                            j + self.rock.x > self.width - 1 or \
                            j + self.rock.x < 0 or \
                            self.field[i + self.rock.y][j + self.rock.x] > 0:
                        intersection = True
        return intersection
    
    
    def new_rock(self):
        self.rock = Rock(self.current_height - HITBOX_SIZE - 3, next(self.rock_cycle))
        
    def freeze(self):
        for i in range(HITBOX_SIZE):
            for j in range(HITBOX_SIZE):
                if i * HITBOX_SIZE + j in self.rock.image():
                    self.field[i + self.rock.y][j + self.rock.x] = 1
        self.n_rocks += 1
        
        self.current_height = min(self.current_height, self.rock.max_y())
        
        if self.current_height < 2*HITBOX_SIZE:
            self.update_field()
            
        if self.n_rocks == self.rock_limit:
            self.state = 'finished'
        else:
            self.new_rock()
            
    def fall_down(self):
        self.rock.y += 1
        if self.intersects():
            self.rock.y -= 1
            self.freeze()

    def go_side(self):
        dx = -1 if next(self.jet_cycle) == '<' else 1
        old_x = self.rock.x

        self.rock.x += dx
        if self.intersects():
            self.rock.x = old_x
    
    
    def get_col_heights(self):
        nz = np.argwhere(self.field)
        lowest_y_for_x = np.zeros(CAVE_WIDTH)
        for x in range(CAVE_WIDTH):
            if x in (cols_w_x := nz[:, 1]):
                lowest_y_for_x[x] = nz[cols_w_x == x][:,0].min()
            else: 
                lowest_y_for_x[x] = self.height
        return lowest_y_for_x
    
    def update_field(self):
        lowest_y_for_x = self.get_col_heights()
        maxy = lowest_y_for_x.max()

        to_roll = int(self.height - maxy)
        self.field = np.roll(self.field, to_roll, axis=0)
        self.field[:to_roll, :] = 0
        
        self.height_outside_of_view += to_roll
        self.current_height += to_roll 
        
    def get_total_height(self):
        return self.height_outside_of_view + self.height - self.current_height

In [91]:
def run_cave_sim(max_height, CAVE_WIDTH, max_rocks, jet_streams):
    c = Cave(max_height, CAVE_WIDTH, max_rocks, jet_streams)
    c.new_rock()
    while not c.state == 'finished':
        c.go_side()
        c.fall_down()
    return c

In [95]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 2022
max_height = 1500*HITBOX_SIZE

jet_streams = get_input('test')

c = run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

In [ ]:
CRT = ''''''
for row in c.field:
    if any(row):
        CRT += '|'
        for pt in row:        
            CRT += '#' if pt else '.'
        CRT += '|\n'
print(CRT)

In [97]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [114]:
CAVE_WIDTH = 7
HITBOX_SIZE = 4
rock_limit = 100000
max_height = 1500*HITBOX_SIZE

jet_streams = get_input('test')

%lprun -f Cave.go_side run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)

Timer unit: 1e-07 s

Total time: 9.33988 s
File: C:\Users\a336214\AppData\Local\Temp\ipykernel_28100\3793717076.py
Function: go_side at line 56

Line #      Hits         Time  Per Hit   % Time  Line Contents
    56                                               def go_side(self):
    57    571421    1622536.0      2.8      1.7          dx = -1 if next(self.jet_cycle) == '<' else 1
    58    571421    1233344.0      2.2      1.3          old_x = self.rock.x
    59                                           
    60    571421    1583484.0      2.8      1.7          self.rock.x += dx
    61    465707   88682216.0    190.4     95.0          if self.intersects():
    62    105714     277251.0      2.6      0.3              self.rock.x = old_x

In [106]:
times = []
for rock_limit in [1, 10, 100, 1000, 10000, 100000]:
    if c in locals():
        del c
    t0 = timer()
    CAVE_WIDTH = 7
    HITBOX_SIZE = 4
    # rock_limit = 1000000000000
    max_height = 1500*HITBOX_SIZE

    jet_streams = get_input('test')

    c = run_cave_sim(max_height, CAVE_WIDTH, rock_limit, jet_streams)
    times.append(timer()-t0)

In [107]:
times

[0.008702499999799329,
 0.0012867000000369444,
 0.00917059999983394,
 0.0826563000000533,
 0.7166383999997379,
 7.264314699999886]

In [ ]:
# 13 years runtime. Probably need to find another way

In [ ]:
CRT = ''''''
for row in c.field:
    if any(row):
        CRT += '|'
        for pt in row:        
            CRT += '#' if pt else '.'
        CRT += '|\n'
print(CRT)

# Part 2